In [1]:
import aesara.tensor as at
import arviz as az
import numpy as np
import pandas as pd
import pymc as pm

In [2]:
rng = np.random.default_rng(1234)

size = 1000
values = rng.normal(size=size)

left_censored = values < -2
interval_censored = np.logical_and(-0.2 < values, values < 0.2)
right_censored = values > 1.8
not_censored = ~np.logical_or.reduce((left_censored, interval_censored, right_censored))

In [3]:
print(sum(left_censored), sum(interval_censored), sum(right_censored), sum(not_censored))
sum(left_censored) + sum(interval_censored) + sum(right_censored) + sum(not_censored)

16 164 48 772


1000

In [4]:
with pm.Model() as model:
    target = 0
    mu = pm.Normal("mu")
    sigma = pm.HalfNormal("sigma")

    # Contribution due to left censored values
    target += np.sum(left_censored) * pm.Normal.logcdf(-2, mu, sigma)

    # Contribution due to right censored values
    target += np.sum(right_censored) * at.log(1 - at.exp(pm.Normal.logcdf(1.8, mu, sigma)))

    # Contribution due to interval censored values
    target += np.sum(interval_censored) * (
        at.log(at.exp(pm.Normal.logcdf(0.2, mu, sigma)) - at.exp(pm.Normal.logcdf(-0.2, mu, sigma)))
    )

    # Contribution due to uncensored values
    likelihood = pm.Normal.dist(mu=mu, sigma=sigma)
    target += at.sum(pm.logp(likelihood, values[not_censored]))

    pm.Potential("y_potential", target)
    idata = pm.sample()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [mu, sigma]


Sampling 2 chains for 1_000 tune and 1_000 draw iterations (2_000 + 2_000 draws total) took 2 seconds.


In [5]:
az.summary(idata)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
mu,0.041,0.033,-0.017,0.103,0.001,0.001,2014.0,1568.0,1.0
sigma,1.017,0.024,0.973,1.061,0.001,0.000,1832.0,1414.0,1.0


In [ ]:
with pm.Model() as model:

    mu = pm.Normal("mu")
    sigma = pm.HalfNormal("sigma")

    y_likelihood = pm.Normal.dist(mu=mu, sigma=sigma)
    y_censored = pm.Censored("y_left", y_likelihood, observed=values)

    idata = pm.sample()